<a href="https://colab.research.google.com/github/namylase/corona_orange/blob/main/Image_detection_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu==2.3.0rc0

     |████████████████████████████████| 320.3MB 56kB/s 
     |████████████████████████████████| 20.1MB 1.4MB/s 
     |████████████████████████████████| 460kB 57.0MB/s 
     |████████████████████████████████| 3.0MB 50.1MB/s 
ERROR: tensorflow 2.4.0 has requirement numpy~=1.19.2, but you'll have numpy 1.18.5 which is incompatible.
ERROR: tensorflow 2.4.0 has requirement tensorboard~=2.4, but you'll have tensorboard 2.2.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0


In [1]:
!git clone https://github.com/hunglc007/tensorflow-yolov4-tflite

Cloning into 'tensorflow-yolov4-tflite'...
remote: Enumerating objects: 780, done.
remote: Total 780 (delta 0), reused 0 (delta 0), pack-reused 780
Receiving objects: 100% (780/780), 197.64 MiB | 32.89 MiB/s, done.
Resolving deltas: 100% (371/371), done.


In [2]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [3]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# weights download
import gdown
url = 'https://drive.google.com/uc?id=1cewMfusmPjYWbrnuJRuKhPMwRe_b9PaT'
output = '/content/tensorflow-yolov4-tflite/data/yolov4.weights'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1cewMfusmPjYWbrnuJRuKhPMwRe_b9PaT
To: /content/tensorflow-yolov4-tflite/data/yolov4.weights
258MB [00:02, 97.8MB/s]


'/content/tensorflow-yolov4-tflite/data/yolov4.weights'

In [5]:
#cudnn

url = 'https://drive.google.com/uc?id=1NLOBQmV6QZpP_c7-693ug0X-bqBpeJLE'
output = '/usr/local/cudnn-10.1-linux-x64-v7.6.5.32 (1).tgz'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1NLOBQmV6QZpP_c7-693ug0X-bqBpeJLE
To: /usr/local/cudnn-10.1-linux-x64-v7.6.5.32 (1).tgz
523MB [00:06, 78.2MB/s]


'/usr/local/cudnn-10.1-linux-x64-v7.6.5.32 (1).tgz'

In [6]:
%cd /usr/local/
!tar -xzvf "cudnn-10.1-linux-x64-v7.6.5.32 (1).tgz"

!chmod a+r /usr/local/cuda/include/cudnn.h
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

%cd /content/tensorflow-yolov4-tflite

/usr/local
cuda/include/cudnn.h
cuda/NVIDIA_SLA_cuDNN_Support.txt
cuda/lib64/libcudnn.so
cuda/lib64/libcudnn.so.7
cuda/lib64/libcudnn.so.7.6.5
cuda/lib64/libcudnn_static.a
#define CUDNN_MAJOR 7
#define CUDNN_MINOR 6
#define CUDNN_PATCHLEVEL 5
--
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

#include "driver_types.h"
/content/tensorflow-yolov4-tflite


In [7]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [8]:
%cd /content/tensorflow-yolov4-tflite
!python save_model.py --weights ./data/yolov4.weights --output ./checkpoints/yolov4-416 --input_size 416 --model yolov4

/content/tensorflow-yolov4-tflite
2021-01-19 08:07:46.308583: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-01-19 08:07:48.105272: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-01-19 08:07:48.112110: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-01-19 08:07:48.112655: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2021-01-19 08:07:48.112688: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-01-19 08:07:48.114240: I tensorflo

In [11]:
!python detect.py --weights ./checkpoints/yolov4-416 --size 416 --model yolov4 --image_folder ./data/0563

2021-01-19 08:11:15.522787: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-01-19 08:11:16.811937: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-01-19 08:11:16.817029: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-01-19 08:11:16.817600: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2021-01-19 08:11:16.817649: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-01-19 08:11:16.822286: I tensorflow/stream_executor/platform/default

In [13]:
!zip -r /content/tensorflow-yolov4-tflite/data/0563.zip /content/tensorflow-yolov4-tflite/data/0563
from google.colab import files
files.download('/content/tensorflow-yolov4-tflite/data/0563.zip')

updating: content/tensorflow-yolov4-tflite/data/0563/ (stored 0%)
updating: content/tensorflow-yolov4-tflite/data/0563/0563_0609.txt (stored 0%)
updating: content/tensorflow-yolov4-tflite/data/0563/0563_0306.txt (stored 0%)
updating: content/tensorflow-yolov4-tflite/data/0563/0563_0606.txt (stored 0%)
updating: content/tensorflow-yolov4-tflite/data/0563/0563_0102.png (deflated 8%)
updating: content/tensorflow-yolov4-tflite/data/0563/0563_0301.txt (stored 0%)
updating: content/tensorflow-yolov4-tflite/data/0563/0563_0205.txt (stored 0%)
updating: content/tensorflow-yolov4-tflite/data/0563/0563_0101.jpg (deflated 0%)
updating: content/tensorflow-yolov4-tflite/data/0563/0563_0509.txt (stored 0%)
updating: content/tensorflow-yolov4-tflite/data/0563/0563_0105.png (deflated 5%)
updating: content/tensorflow-yolov4-tflite/data/0563/0563_0206.png (deflated 20%)
updating: content/tensorflow-yolov4-tflite/data/0563/0563_0109.png (deflated 11%)
updating: content/tensorflow-yolov4-tflite/data/0563/

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>